<a href="https://www.kaggle.com/code/gauravjains2003/nasa-near-earth-objects?scriptVersionId=91243233" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import matplotlib.pyplot as plt
import ipywidgets as widgets
from bokeh.plotting import figure,show,output_file,output_notebook,curdoc
from bokeh.models import Select,ColumnDataSource,Spinner
from bokeh.layouts import column,row
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
import math
output_notebook()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Reading Data from NASA's Near Earth Objects API , run the below code to download data in JSON format#

'''
response = requests.get("https://api.nasa.gov/neo/rest/v1/neo/browse?api_key=DEMO_KEY")
data = response.json()

#Writing API data in JSON File#

with open('data.json', 'w') as outfile:
     json.dump(data, outfile, sort_keys = True, indent = 4)

'''

#Reading Data from the downloaded JSON data file#

json_file_path = "/kaggle/input/nasaneo/data.json"
with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())

In [ ]:
#Creating Empty DataFrame with just column names#

col = ['Name','Designation','ID','Potentially_Hazardous','Estimated_Diam._Max(KM)','Estimated_Diam._Min(KM)','Magnitude','Inclination','First_Observation_Date','Last_Observation_Date','Close_Approach_Date','Miss_Distance(KM)','Relative_Velocity(KM/H)','Orbiting_body']
df = pd.DataFrame(columns = col)
print(df)

In [ ]:
#Looping through the contents json string to extract meaningful data#

for i in range(0,20):
  
  #Reading main parameters in main_list#
  
  main_list = []

  main_list.append(contents['near_earth_objects'][i]['name'])
 
  main_list.append(contents['near_earth_objects'][i]['designation'])
 
  main_list.append(contents['near_earth_objects'][i]['id'])
  
  main_list.append(contents['near_earth_objects'][i]['is_potentially_hazardous_asteroid'])
  
  main_list.append(contents['near_earth_objects'][i]['estimated_diameter']['kilometers']['estimated_diameter_max'])
  
  main_list.append(contents['near_earth_objects'][i]['estimated_diameter']['kilometers']['estimated_diameter_min'])
  
  main_list.append(contents['near_earth_objects'][i]['absolute_magnitude_h'])
  
  main_list.append(contents['near_earth_objects'][i]['orbital_data']['inclination'])
  
  main_list.append(contents['near_earth_objects'][i]['orbital_data']['first_observation_date'])
  
  main_list.append(contents['near_earth_objects'][i]['orbital_data']['last_observation_date'])
  
  
  for j in range(0,len(contents['near_earth_objects'][i]['close_approach_data'])):
    
    #Reading sub parameters (Close Approach Data) in sub_list#
    
    sub_list = []
    
    sub_list.append(contents['near_earth_objects'][i]['close_approach_data'][j]['close_approach_date'])
    
    sub_list.append(contents['near_earth_objects'][i]['close_approach_data'][j]['miss_distance']['kilometers'])
    
    sub_list.append(contents['near_earth_objects'][i]['close_approach_data'][j]['relative_velocity']['kilometers_per_hour'])
    
    sub_list.append(contents['near_earth_objects'][i]['close_approach_data'][j]['orbiting_body'])
    
       
    #Creating final list ( a row of data) and inserting it as a row in dataframe created earlier #
    
    final_list = main_list + sub_list
    
    temp_df = pd.DataFrame([final_list],columns=df.columns)
    
    df=pd.concat([df,temp_df],ignore_index=True)

In [ ]:
df.info()

In [ ]:
#Creating new column which only contains the year valye of Close Approach Year#
df['Close_Approach_Year'] = pd.to_datetime(df['Close_Approach_Date']).dt.year
df.info()

In [ ]:
#Code for first figure - A scatter plot showing decades and count of close-approaches in that decade#


#Extracting deacdes from Close Approach Year column in dataframe#
decade_list = []

for year in df["Close_Approach_Year"].tolist():
    decade = (year//10)*10
    decade_list.append(decade)

#Creating a new column in dataframe with decade information#    
df["Close_Approach_Decade"]= decade_list

#Getting unique decades , sorting and converting them to a list of strings for graph plotting#
unique_decades = list(set(decade_list))
unique_decades.sort()
unique_decades = [str(x) for x in unique_decades]

#Extracting count of close approaches pe rdecade# 
dec_count_list= df.groupby("Close_Approach_Decade").size().tolist()


   
TOOLS = "pan,wheel_zoom,reset,save"
fig1 = figure(width=500,height=500,y_range=unique_decades, title="Close Approaches - Decade",x_axis_label="Close_Approach_Count",y_axis_label="Decades",tools=TOOLS)
fig1.hover.point_policy = "follow_mouse"

#Scatter Plot#
points=fig1.scatter(dec_count_list,unique_decades)

spinner = Spinner(title="Glyph size", low=1, high=40, step=0.5, value=4, width=80)
spinner.js_link('value', points.glyph, 'size')

#Use below if you want horizontal bar plot#
'''
fig1.hbar(y = unique_decades, height = 0.5, left = 0, right = dec_count_list)
fig1.xgrid.grid_line_color = None
'''

#fig1.xgrid.grid_line_color = None
fig1.x_range.start = 0

show(fig1)


In [ ]:
df

In [ ]:
#Identfying hazardous/non-hazardous near earth objects close approaches#

hazas = df.loc[df['Potentially_Hazardous'] == True]['Name'].unique().tolist()
non_hazas = df.loc[df['Potentially_Hazardous'] == False]['Name'].unique().tolist()

print(hazas)
print("-----------")
print(non_hazas)

In [ ]:
#Code for second figure - A pie plot showing hazardous/non-hazardous close approaches near earth objects#

TOOLS = "pan,wheel_zoom,reset,save"

total = len(hazas) + len(non_hazas)
hazas_pct = (len(hazas)/total)*100
non_hazas_pct = (len(non_hazas)/total)*100
             
# name of the fields 
fields = ["Hazardous", "Non-Hazardous"]
    
# % tage weightage  
percentages = [hazas_pct,non_hazas_pct]  

# then, we will instantiate the figure object   
fig2 = figure(title = "Hazardous vs Non-Hazardous Close Approaches",height=400,width=400,tools=TOOLS)

# formula for converting percentage into radians  
radians1 = [math.radians((percent / 100) * 360) for percent in percentages]  
    
# Generating the start angle values  
start_angle = [math.radians(0)]  
prev = start_angle[0]  
for k in radians1[:-1]:  
    start_angle.append(k + prev)  
    prev = k + prev  
    
# generating the end angle values  
end_angle = start_angle[1:] + [math.radians(0)]  
    
# initiate the center of the pie chart  
x = 0  
y = 0  
    
# then, we will initiate the radius of the glyphs  
radius = 1  
    
# now, generate the color of the wedges  
color1 = ["red","green"]  
    
# now, we will plot the graph  
for k in range(len(fields)):  
    fig2.wedge(x, y, radius,  
                start_angle = start_angle[k],  
                end_angle = end_angle[k],  
                color = color1[k],  
                legend_label = str(fields[k])+" - "+str(percentages[k])+"%")

fig2.legend.location = "top_right"

# At last, we will display the graph  
show(fig2)  

In [ ]:
#Creating a bokeh data-table with selected columns from the dataframe data #

data = dict(df[['Name','Designation','Potentially_Hazardous','First_Observation_Date','Last_Observation_Date','Close_Approach_Date','Orbiting_body']])
source = ColumnDataSource(data)

columns = [
        TableColumn(field="Name", title="Name"),
        TableColumn(field="Designation", title="Designation"),
        TableColumn(field="Potentially_Hazardous", title="Potentially_Hazardous"),
        TableColumn(field="Orbiting_body", title="Orbiting_body"),
        TableColumn(field="First_Observation_Date", title="First_Observation_Date"),
        TableColumn(field="Last_Observation_Date", title="Last_Observation_Date"),
        TableColumn(field="Close_Approach_Date", title="Close_Approach_Date")
    ]
data_table = DataTable(source=source, columns=columns, width=500, height=400)

show(data_table)

In [ ]:
#Getting Unique Near Earth Objects [NEO's]#

uniq_names = df['Name'].unique().tolist()

uniq_names

In [ ]:
# Code to draw bar graph for Close_Approach_Date and Miss_Distance - kind(Matplotlib or Bokeh) #

def draw_bar_plot(name,kind):
  
  df_temp = df.loc[df['Name'] == name]
  Close_Approach_Date_List = df_temp['Close_Approach_Date'].tolist()
  Miss_Distance_KM__List = df_temp['Miss_Distance(KM)'].tolist()
  Miss_Distance_KM__List = [float(j) for j in Miss_Distance_KM__List]
  
  if(kind=="matplotlib"):
    
    fig = plt.figure(figsize=(14,10))
    ax = fig.add_subplot(1,1,1)
    ax.barh(Close_Approach_Date_List,Miss_Distance_KM__List)
    ax.bar_label(ax.containers[0])
    ax.set(title="Close_Approach_Date vs Miss_DIstance_KM",xlabel="Miss_Distance_KM",ylabel="Close_Approach_Dates")
    plt.tight_layout()
    plt.show()
    #plt.savefig(name+".png")
    
  elif(kind=="bokeh"):
        
    bar_chart = figure(y_range=Close_Approach_Date_List,title="Close_Approach_Date vs Miss_DIstance_KM",x_axis_label="Miss_Distance_KM",y_axis_label="Close_Approach_Dates")
    bar_chart.hbar(y = Close_Approach_Date_List, height = 0.5, left = 0, right = Miss_Distance_KM__List)
    #output_file(i+".html")
    bar_chart.xgrid.grid_line_color = None
    bar_chart.x_range.start = 0
    show(bar_chart)

In [ ]:
#Uncomment and Run the below code if you need individual graphs for all the NEOs #

'''
#Loop to go through all the data of each NEO and create bar graphs #
for name in uniq_names:
    
    #pass matplotlib or bokeh in 2nd argument for type of chart#
    draw_bar_plot(name,"matplotlib")
  
'''

In [ ]:
#Function to be used when using widgets.interact , to create interactive graphs within notebook #########


def miss_dist(NEO):
  df_temp = df.loc[df['Name'] == NEO]
  Close_Approach_Date_List = df_temp['Close_Approach_Date'].tolist()
  Miss_Distance_KM__List = df_temp['Miss_Distance(KM)'].tolist()
  Miss_Distance_KM__List = [float(j) for j in Miss_Distance_KM__List]
  
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1,1,1)
  ax.barh(Close_Approach_Date_List,Miss_Distance_KM__List)
  ax.bar_label(ax.containers[0])
  ax.set(title="Close_Approach_Date vs Miss_DIstance_KM",xlabel="Miss_Distance_KM",ylabel="Close_Approach_Dates")
  plt.tight_layout()
  plt.show()
  #plt.savefig(NEO+".png")

widgets.interact(miss_dist,NEO=uniq_names)


##############################################################

In [ ]:
# Bokeh Dashboard - Use the below code with command "bokeh serve --show <<filename.py>>" # 

def update_bar_chart(attrname, old, new):
  
  
  df_temp = df.loc[df['Name'] == select1.value]
  Close_Approach_Date_List = df_temp['Close_Approach_Date'].tolist()
  Miss_Distance_KM__List = df_temp['Miss_Distance(KM)'].tolist()
  Miss_Distance_KM__List = [float(j) for j in Miss_Distance_KM__List]

  bar_chart = figure(width=500,height=500,y_range=Close_Approach_Date_List,title="Close_Approach_Date vs Miss_DIstance_KM",x_axis_label="Miss_Distance_KM",y_axis_label="Close_Approach_Dates")
  bar_chart.hbar(y = Close_Approach_Date_List, height = 0.5, left = 0, right = Miss_Distance_KM__List)
  layout_with_widgets.children[0].children[0].children[1] = bar_chart


#Displaying default chart #

df_temp = df.loc[df['Name'] == uniq_names[0]]
Close_Approach_Date_List = df_temp['Close_Approach_Date'].tolist()
Miss_Distance_KM__List = df_temp['Miss_Distance(KM)'].tolist()
Miss_Distance_KM__List = [float(j) for j in Miss_Distance_KM__List]


fig3 = figure(width=500,height=500,y_range=Close_Approach_Date_List,title="Close_Approach_Date vs Miss_DIstance_KM",x_axis_label="Miss_Distance_KM",y_axis_label="Close_Approach_Dates")
fig3.hbar(y = Close_Approach_Date_List, height = 0.5, left = 0, right = Miss_Distance_KM__List)
output_file('NASA NEO Dashboard.html')
fig3.xgrid.grid_line_color = None
fig3.x_range.start = 0
#fig.xaxis.major_label_orientation = 1.2

select1 = Select(title="Choose NEO",options=uniq_names,value=uniq_names[0],width=200)

select1.on_change("value", update_bar_chart)

layout_with_widgets = column(
                        row(
                            column(select1,fig3),
                            column(spinner,fig1) ),
                            row(data_table,fig2),
                            
)

show(layout_with_widgets)

curdoc().add_root(layout_with_widgets)

#bokeh serve --show main.py



##########################################################